In [1]:
import pandas as pd
import pandas_datareader.data as web
import datetime
import backtrader as bt
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pyfolio as pf
import quantstats
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)
import sys
import time
import sqlite3
from scipy.stats import rankdata
from scipy.stats import stats
from scipy.optimize import minimize

C:\ProgramData\Anaconda3\lib\site-packages\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


만들어 두었던 미국 상장 ETF 가격 데이터 DB에서 ticker 목록을 가져옵니다. 필터링 예시로는 거래량과 거래된 기간을(ETF의 연령) 들 수 있습니다. 거래 체결을 위해서는 적절한 수준의 유동성이 보장되어야 합니다. 최근 50일 평균 거래량이(volume) 5만 주 이상인 것들로 제한하겠습니다. 또한, 상장 후 5년(1260 거래일) 이상 된 ETF들로 구성하겠습니다.

In [2]:
con = sqlite3.connect('USListedETFPrice.db')
ticker = pd.read_sql("select name from sqlite_master where type = 'table'; ", con)

In [3]:
def UniverseFilter(ticker):
    
    volume_list = []
    
    for i in range(len(ticker)):
        temp_df = pd.read_sql("select volume from '"+str(ticker.iloc[i,0])+"'", con)
        avg = np.mean(temp_df['Volume'][-50:])
        volume_list.append(avg)
        
    ticker['volume'] = volume_list
    ticker_liquid = ticker[ticker['volume']>=50000]
    ticker_liquid = ticker_liquid.reset_index(drop=True)
    
    age_list = []
    
    for i in range(len(ticker_liquid)):
        temp_age = pd.read_sql("select count(*) from '"+str(ticker_liquid.iloc[i,0])+"'", con)
        age = temp_age.iloc[0,0]
        age_list.append(age)
    
    ticker_liquid['age'] = age_list
    ticker_final = ticker_liquid[ticker_liquid['age']>=1260]
    ticker_final = ticker_final.reset_index(drop=True)

    return ticker_final

In [4]:
UniverseFilter(ticker)

,name,volume,age
0,AAXJ,1187052.0,3150
1,ACWI,3494860.0,3248
2,ACWX,1182516.0,3246
3,AIA,299882.0,3335
4,ANGL,1882164.0,2230
...,...,...,...
712,XTN,98256.0,2533
713,YANG,384642.0,2822
714,YINN,1487910.0,2822
715,YYY,147868.0,2187


In [ ]:
temp = UniverseFilter(ticker)
temp.to_excel('UniverseFilter.xlsx')